In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [2]:
#data
df=pd.read_csv('C:\Kate\Python\Authorship Attribution\data\AllTweets.csv')
author1='KimKardashian'
author2='HillaryClinton'
df_kk=df.loc[(df['author'] == author1)]
df_hc=df.loc[(df['author'] == author2)]
df=df_kk.append(df_hc,ignore_index=True)
len(df)

14044

In [3]:
import random
#2000 random sample rows for KK
rows = random.sample(df_kk.index, 500)
df_kk = df_kk.ix[rows]
#2000 random sample rows for HC
rows = random.sample(df_hc.index, 500)
df_hc = df_hc.ix[rows]
#join back together
df=df_kk.append(df_hc,ignore_index=True)
len(df)

1000

In [4]:
#data pre-processing
df.drop(df[df.retweet==True].index, inplace=True)
df['num_of_words'] = df["text"].str.split().apply(len)
df.drop(df[df.num_of_words<4].index, inplace=True)
df["text"].replace(r"http\S+", "URL", regex=True,inplace=True)
df["text"].replace(r"@\S+", "REF", regex=True ,inplace=True)
df["text"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"\d+", "NUM", regex=True,inplace=True)
len(df)

950

In [5]:
#http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
pos_code_map={'CC':'A','CD':'B','DT':'C','EX':'D','FW':'E','IN':'F','JJ':'G','JJR':'H','JJS':'I','LS':'J','MD':'K','NN':'L','NNS':'M',
'NNP':'N','NNPS':'O','PDT':'P','POS':'Q','PRP':'R','PRP$':'S','RB':'T','RBR':'U','RBS':'V','RP':'W','SYM':'X','TO':'Y','UH':'Z',
'VB':'1','VBD':'2','VBG':'3','VBN':'4','VBP':'5','VBZ':'6','WDT':'7','WP':'8','WP$':'9','WRB':'@'}
code_pos_map={v: k for k, v in pos_code_map.iteritems()}
#Python 3 inv_map = {v: k for k, v in my_map.items()}

In [6]:
def convert(tag):
    try:
        code=pos_code_map[tag]
    except:
        code='?'
    return code
def inv_convert(code):
    try:
        tag=code_pos_map[code]
    except:
        tag='?'
    return tag

In [7]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag, word_tokenize
def pos_tags(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    return "".join(convert(tag) for (word, tag) in nltk.pos_tag(text_processed))
def text_pos_inv_convert(text):
    return "-".join(inv_convert(c.upper()) for c in text)

In [8]:
df['text_pos']=df.apply(lambda x: pos_tags(x['text']), axis=1)

In [9]:
df.ix[:,['author','text','text_pos']].head()

,author,text,text_pos
0,KimKardashian,Happy birthday Khloball!!! URL #HappyBirthday...,GLNNN
1,KimKardashian,"I'm soooo sleepy, too lazy to get up and wash ...",R5GLTGY1WA1S1WTA1SMAR5Y1
2,KimKardashian,My message to REF on her NUMth birthday! URL ...,SLY1FRNLNN
4,KimKardashian,East coast Keeping Up With the Kardashians: Ab...,NL3WFCMFNMTFNR53FR5
5,KimKardashian,Such a tremendous honor to be on the cover of ...,GCGLY1FCLFNRT2CK11SNK1GLLL


In [10]:
df_features=pd.DataFrame()

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
for a in df.author.unique():
    v = CountVectorizer(analyzer='char',ngram_range=(7, 7))
    ngrams = v.fit_transform(df[df['author'] == a]['text_pos'])
    df_t=pd.DataFrame(
    {'Feature': v.get_feature_names(),
     'Count': list(ngrams.sum(axis=0).flat),
     'Author': a
    })
    #
    df_features=df_features.append(df_t,ignore_index=True)

In [12]:
df_features['Feature_POS']=df_features.apply(lambda x: text_pos_inv_convert(x['Feature']), axis=1)

In [13]:
df_features[~df_features.Feature.isin(df_features[df_features['Author'] != author2].Feature)].sort_values('Count', ascending=False).ix[:,['Author','Count','Feature','Feature_POS']].head()

,Author,Count,Feature,Feature_POS
4839,HillaryClinton,3,4fclfnn,VBN-IN-DT-NN-IN-NNP-NNP
9264,HillaryClinton,3,nnnllll,NNP-NNP-NNP-NN-NN-NN-NN
9236,HillaryClinton,3,nnn2cgl,NNP-NNP-NNP-VBD-DT-JJ-NN
7962,HillaryClinton,3,lk1cglf,NN-MD-VB-DT-JJ-NN-IN
9560,HillaryClinton,3,r5cl8k1,PRP-VBP-DT-NN-WP-MD-VB


In [14]:
df_features[~df_features.Feature.isin(df_features[df_features['Author'] != author1].Feature)].sort_values('Count', ascending=False).ix[:,['Author','Count','Feature','Feature_POS']].head()

,Author,Count,Feature,Feature_POS
2025,KimKardashian,4,lfnnnnn,NN-IN-NNP-NNP-NNP-NNP-NNP
1726,KimKardashian,3,gnnr5r5,JJ-NNP-NNP-PRP-VBP-PRP-VBP
1265,KimKardashian,3,fco6fnn,IN-DT-NNPS-VBZ-IN-NNP-NNP
3975,KimKardashian,3,wfco6fn,RP-IN-DT-NNPS-VBZ-IN-NNP
3164,KimKardashian,3,nnr5r5g,NNP-NNP-PRP-VBP-PRP-VBP-JJ


In [15]:
from sklearn.cross_validation import train_test_split
twt_train, twt_test, author_train, author_test = train_test_split(df.ix[:,['text','text_pos']], df['author'], test_size=0.3, random_state=42)

In [16]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
def text_process(text):
    """
    Takes in a string of text, then performs the following:
    1. Tokenizes and removes punctuation
    3. Stems
    4. Returns a list of the cleaned text
    """

    # tokenizing
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    
    # steming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    

    return text_processed

In [17]:
ScoreSummaryByModelParams = list()

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [20]:
class TextAndTextCodedExtractor(BaseEstimator, TransformerMixin):
    """Extract the text & text_pos from a tweet in a single pass.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, tweets):
        features=tweets.ix[:,['text_pos','text']].to_records(index=False)

        return features

In [21]:
def ModelParamsEvaluation (f_union,model,params,comment):
    pipeline = Pipeline([
    # Extract the text & text_coded
    ('textandtextcoded', TextAndTextCodedExtractor()),

    # Use FeatureUnion to combine the features from text and text_coded
    ('union', f_union, ),

    # Use a  classifier on the combined features
    ('clf', model),
    ])
    grid_search = GridSearchCV(estimator=pipeline, param_grid=params, verbose=1, cv=10)
    grid_search.fit(twt_train, author_train)
    #best score
    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(params.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        ScoreSummaryByModelParams.append([comment,grid_search.best_score_,"\t%s: %r" % (param_name, best_parameters[param_name])])    
 

In [22]:
f1_union=FeatureUnion(
        transformer_list=[
              # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char')),
            ])),               

        ],
    )

In [23]:
from sklearn.naive_bayes import BernoulliNB
p = {
    'union__char__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__char__tfidf__ngram_range': ((2, 2), (3, 3)), 
    'union__char__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char')

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    2.9s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   12.5s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   28.7s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:   53.3s
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  1.4min


Best score: 0.953
Best parameters set:
	clf__alpha: 0.1
	union__char__tfidf__max_df: 0.5
	union__char__tfidf__max_features: None
	union__char__tfidf__ngram_range: (3, 3)


[Parallel(n_jobs=1)]: Done 1440 out of 1440 | elapsed:  1.6min finished


In [24]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf',    TfidfVectorizer(analyzer='word')),
            ])),              

        ],
    )

In [25]:
p = {
    'union__word__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__word__tfidf__ngram_range': ((1, 1),(2, 2), (3, 3),(4,4),(5,5)), 
    'union__word__tfidf__max_features': (None, 5000, 10000, 50000),
    'union__word__tfidf__stop_words': (None, 'english'),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, word')

Fitting 10 folds for each of 720 candidates, totalling 7200 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    2.3s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   10.1s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   23.4s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:   42.3s
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  1.1min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  1.6min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed:  2.1min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed:  2.8min
[Parallel(n_jobs=1)]: Done 4049 tasks       | elapsed:  3.6min
[Parallel(n_jobs=1)]: Done 4999 tasks       | elapsed:  4.5min
[Parallel(n_jobs=1)]: Done 6049 tasks       | elapsed:  5.4min


Best score: 0.934
Best parameters set:
	clf__alpha: 0.1
	union__word__tfidf__max_df: 0.5
	union__word__tfidf__max_features: None
	union__word__tfidf__ngram_range: (1, 1)
	union__word__tfidf__stop_words: None


[Parallel(n_jobs=1)]: Done 7199 tasks       | elapsed:  6.5min
[Parallel(n_jobs=1)]: Done 7200 out of 7200 | elapsed:  6.5min finished


In [26]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling word features from the text
            ('text', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process)),
            ])),              

        ],
    )

In [27]:
p = {
    'union__text__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__text__tfidf__ngram_range': ((1, 1),(2, 2), (3, 3),(4,4),(5,5)), 
    'union__text__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, stemmed words, no stop words')

Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    7.0s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   28.9s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:  1.1min
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  1.9min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  3.0min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  4.3min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed:  5.9min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed:  7.7min


Best score: 0.944
Best parameters set:
	clf__alpha: 0.001
	union__text__tfidf__max_df: 0.5
	union__text__tfidf__max_features: None
	union__text__tfidf__ngram_range: (1, 1)


[Parallel(n_jobs=1)]: Done 3600 out of 3600 | elapsed:  8.6min finished


In [28]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling pos tag features  from the text_pos
            ('text_pos', Pipeline([
            ('selector', ItemSelector(key='text_pos')),
            ('tfidf',    TfidfVectorizer(analyzer='char')),
            ])),                  

        ],
    )

In [29]:
p = {
    'union__text_pos__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__text_pos__tfidf__ngram_range': ((3, 3), (4, 4),(5,5),(6,6),(7,7)), 
    'union__text_pos__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, POS tags')

Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    1.9s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:    8.0s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   18.1s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:   32.4s
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:   51.0s
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  1.2min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed:  1.7min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed:  2.2min


Best score: 0.716
Best parameters set:
	clf__alpha: 0.1
	union__text_pos__tfidf__max_df: 0.5
	union__text_pos__tfidf__max_features: None
	union__text_pos__tfidf__ngram_range: (3, 3)


[Parallel(n_jobs=1)]: Done 3600 out of 3600 | elapsed:  2.4min finished


In [30]:
df_ScoreSummaryByModelParams=DataFrame(ScoreSummaryByModelParams,columns=['Method','BestScore','BestParameter'])
df_ScoreSummaryByModelParams.sort_values(['BestScore'],ascending=False,inplace=True)
df_ScoreSummaryByModelParams

,Method,BestScore,BestParameter
0,"Bernoulli Naive Bayes, char",0.953383,\tclf__alpha: 0.1
1,"Bernoulli Naive Bayes, char",0.953383,\tunion__char__tfidf__max_df: 0.5
2,"Bernoulli Naive Bayes, char",0.953383,\tunion__char__tfidf__max_features: None
3,"Bernoulli Naive Bayes, char",0.953383,"\tunion__char__tfidf__ngram_range: (3, 3)"
12,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.944361,"\tunion__text__tfidf__ngram_range: (1, 1)"
11,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.944361,\tunion__text__tfidf__max_features: None
10,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.944361,\tunion__text__tfidf__max_df: 0.5
9,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.944361,\tclf__alpha: 0.001
8,"Bernoulli Naive Bayes, word",0.933835,\tunion__word__tfidf__stop_words: None
7,"Bernoulli Naive Bayes, word",0.933835,"\tunion__word__tfidf__ngram_range: (1, 1)"


In [31]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [32]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f2_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    4.6s


Best score: 0.953
Best parameters set:
	clf__alpha: 0.0001


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    5.7s finished


In [33]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling stememd word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [34]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f2_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + stemmed word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    8.9s


Best score: 0.958
Best parameters set:
	clf__alpha: 0.1


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   10.9s finished


In [35]:
f3_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),    
            # Pipeline for pulling word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [36]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + stemmed word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   10.3s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   12.6s finished


Best score: 0.956
Best parameters set:
	clf__alpha: 0.0001


In [37]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [38]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + stemmed word + POS tags')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    9.8s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   12.1s finished


Best score: 0.952
Best parameters set:
	clf__alpha: 0.1


In [39]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [40]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + POS tags')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    5.6s


Best score: 0.949
Best parameters set:
	clf__alpha: 0.1


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    6.9s finished


In [41]:
f4_union=FeatureUnion(
        transformer_list=[

            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
             # Pipeline for pulling word features after word_processing from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [42]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f4_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + stemmed word + POS tag')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   11.1s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   13.6s finished


Best score: 0.953
Best parameters set:
	clf__alpha: 0.1


In [43]:
df_ScoreSummaryByModelParams=DataFrame(ScoreSummaryByModelParams,columns=['Method','BestScore','BestParameter'])
df_ScoreSummaryByModelParams.sort_values(['BestScore'],ascending=False,inplace=True)
df_ScoreSummaryByModelParams

,Method,BestScore,BestParameter
18,"Bernoulli Naive Bayes, char + stemmed word",0.957895,\tclf__alpha: 0.1
19,"Bernoulli Naive Bayes, char + word + stemmed word",0.956391,\tclf__alpha: 0.0001
0,"Bernoulli Naive Bayes, char",0.953383,\tclf__alpha: 0.1
1,"Bernoulli Naive Bayes, char",0.953383,\tunion__char__tfidf__max_df: 0.5
17,"Bernoulli Naive Bayes, char + word",0.953383,\tclf__alpha: 0.0001
22,"Bernoulli Naive Bayes, char + word + stemmed w...",0.953383,\tclf__alpha: 0.1
2,"Bernoulli Naive Bayes, char",0.953383,\tunion__char__tfidf__max_features: None
3,"Bernoulli Naive Bayes, char",0.953383,"\tunion__char__tfidf__ngram_range: (3, 3)"
20,"Bernoulli Naive Bayes, char + stemmed word + P...",0.951880,\tclf__alpha: 0.1
21,"Bernoulli Naive Bayes, char + word + POS tags",0.948872,\tclf__alpha: 0.1


In [44]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc,precision_score, accuracy_score, recall_score, f1_score
from scipy import interp

In [45]:
ScoreSummaryByVector = list()

In [46]:
def PredictionEvaluation(author_test_b,author_predictions_b,comment):
    Precision=precision_score(author_test_b,author_predictions_b)
    print ('Precision: %0.3f' % (Precision))
    Accuracy=accuracy_score(author_test_b,author_predictions_b)
    print ('Accuracy: %0.3f' % (Accuracy))
    Recall=recall_score(author_test_b,author_predictions_b)
    print ('Recall: %0.3f' % (Recall))
    F1=f1_score(author_test_b,author_predictions_b)
    print ('F1: %0.3f' % (F1))
    print ('Confussion matrix:')
    print (confusion_matrix(author_test_b,author_predictions_b))
    ROC_AUC=roc_auc_score(author_test_b,author_predictions_b)
    print ('ROC-AUC: %0.3f' % (ROC_AUC))
    ScoreSummaryByVector.append([Precision,Accuracy,Recall,F1,ROC_AUC,comment])

In [47]:
def ModelRun (f_union,model):
    pipeline = Pipeline([
    # Extract the text & text_coded
    ('textandtextcoded', TextAndTextCodedExtractor()),

    # Use FeatureUnion to combine the features from text and text_coded
    ('union', f_union, ),

    # Use a  classifier on the combined features
    ('clf', model),
    ])
    pipeline.fit(twt_train, author_train)
    author_predicted = pipeline.predict(twt_test)
    
    feature_names=list()
    for p in (pipeline.get_params()['union'].transformer_list):
        fn=(p[0],pipeline.get_params()['union'].get_params()[p[0]].get_params()['tfidf'].get_feature_names())
        feature_names.append(fn)
    df_fn=pd.DataFrame()
    for fn in feature_names:
        df_fn= df_fn.append(pd.DataFrame(
        {'FeatureType': fn[0],
         'Feature': fn[1]
        }),
        ignore_index=True)    
    
    from sklearn.preprocessing import LabelBinarizer
    lb = LabelBinarizer()
    author_test_b = lb.fit_transform(author_test.values)
    author_predicted_b  = lb.fit_transform(author_predicted)
    return (df_fn,pipeline.get_params()['clf'],author_predicted,author_predicted_b, author_test_b)

In [48]:
def most_informative_feature_for_binary_classification(feature_names, classifier):
    class_labels = classifier.classes_

    topnvalues_class0 = sorted(zip(classifier.coef_[0], feature_names['Feature'].values, feature_names['FeatureType'].values))
    topnvalues_class1 = sorted(zip(classifier.coef_[0], feature_names['Feature'].values, feature_names['FeatureType'].values), reverse=True)

    topn_df_class0=pd.DataFrame(topnvalues_class0, columns=['Coef','Feature','FeatureType'])
    topn_df_class0['Author']=class_labels[0]
    
    topn_df_class1=pd.DataFrame(topnvalues_class1, columns=['Coef','Feature','FeatureType'])
    topn_df_class1['Author']=class_labels[1]    
    
    topn_df=topn_df_class0.append(topn_df_class1)
    
        
    return topn_df

In [49]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling stememd word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [50]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f2_union,BernoulliNB(alpha=0.0001))

In [51]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+stemmed word')

Precision: 0.964
Accuracy: 0.961
Recall: 0.957
F1: 0.961
Confussion matrix:
[[140   5]
 [  6 134]]
ROC-AUC: 0.961


In [52]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [53]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f3_union,BernoulliNB(alpha=0.0001))

In [54]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+stemmed word+POS tag')

Precision: 0.957
Accuracy: 0.954
Recall: 0.950
F1: 0.953
Confussion matrix:
[[139   6]
 [  7 133]]
ROC-AUC: 0.954


In [55]:
f4_union=FeatureUnion(
        transformer_list=[

            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
             # Pipeline for pulling word features after word_processing from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [56]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f4_union,BernoulliNB(alpha=0.0001))

In [57]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+word+stemmed word+POS tag')

Precision: 0.942
Accuracy: 0.954
Recall: 0.963
F1: 0.953
Confussion matrix:
[[141   8]
 [  5 131]]
ROC-AUC: 0.955


In [58]:
df_ScoreSummaryByVector=DataFrame(ScoreSummaryByVector,columns=['Precision','Accuracy','Recall','F1','ROC-AUC','Vector'])
df_ScoreSummaryByVector.sort_values(['F1'],ascending=False,inplace=True)
df_ScoreSummaryByVector

,Precision,Accuracy,Recall,F1,ROC-AUC,Vector
0,0.964029,0.961404,0.957143,0.960573,0.961330,char+stemmed word
1,0.956835,0.954386,0.950000,0.953405,0.954310,char+stemmed word+POS tag
2,0.942446,0.954386,0.963235,0.952727,0.954772,char+word+stemmed word+POS tag


In [59]:
TopFeatures_df=most_informative_feature_for_binary_classification(feature_names, clf)

In [60]:
df1=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='char')),['Author','Coef','Feature']].head(10)
df1.rename(columns={'Coef':'CoefChar','Feature':'Char'}, inplace=True)
df1.reset_index(inplace=True)
df2=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='word')),['Coef','Feature']].head(10)
df2.rename(columns={'Coef':'CoefWord','Feature':'Word'}, inplace=True)
df2.reset_index(inplace=True)
df3=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='text')),['Coef','Feature']].head(10)
df3.rename(columns={'Coef':'CoefText','Feature':'Text'}, inplace=True)
df3.reset_index(inplace=True)
df4=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='text_pos')),['Coef','Feature']].head(10)
df4.rename(columns={'Coef':'CoefTextPOS','Feature':'TextPOS'}, inplace=True)
df4['TextPOS']=df4.apply(lambda x: text_pos_inv_convert(x['TextPOS']), axis=1)
df4.reset_index(inplace=True)
df_kk_top_features = pd.concat([df1,df2,df3,df4],axis=1)
df_kk_top_features.drop('index', axis=1, inplace=True)
df_kk_top_features

,Author,CoefChar,Char,CoefWord,Word,CoefText,Text,CoefTextPOS,TextPOS
0,HillaryClinton,-14.984893,\nhi,-14.984893,abandon,-14.984893,_,-14.984893,VB-VB-VB
1,HillaryClinton,-14.984893,\nnu,-14.984893,abhorrent,-14.984893,abandon,-14.984893,VB-VB-WP
2,HillaryClinton,-14.984893,\nto,-14.984893,abiding,-14.984893,abhorr,-14.984893,VB-VB-CD
3,HillaryClinton,-14.984893,\n✓,-14.984893,able,-14.984893,abid,-14.984893,VB-VB-IN
4,HillaryClinton,-14.984893,"""d",-14.984893,abortion,-14.984893,abl,-14.984893,VB-VB-NN
5,HillaryClinton,-14.984893,"""h",-14.984893,abortions,-14.984893,abort,-14.984893,VB-VB-NNP
6,HillaryClinton,-14.984893,"""t",-14.984893,aca,-14.984893,aca,-14.984893,VB-VB-RB
7,HillaryClinton,-14.984893,"""w",-14.984893,accept,-14.984893,accept,-14.984893,VB-VBG-DT
8,HillaryClinton,-14.984893,#d,-14.984893,account,-14.984893,account,-14.984893,VB-VBG-JJ
9,HillaryClinton,-14.984893,#e,-14.984893,achieve,-14.984893,achiev,-14.984893,VB-VBG-JJR


In [61]:
df1=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='char')),['Author','Coef','Feature']].head(10)
df1.rename(columns={'Coef':'CoefChar','Feature':'Char'}, inplace=True)
df1.reset_index(inplace=True)
df2=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='word')),['Coef','Feature']].head(10)
df2.rename(columns={'Coef':'CoefWord','Feature':'Word'}, inplace=True)
df2.reset_index(inplace=True)
df3=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='text')),['Coef','Feature']].head(10)
df3.rename(columns={'Coef':'CoefText','Feature':'Text'}, inplace=True)
df3.reset_index(inplace=True)
df4=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='text_pos')),['Coef','Feature']].head(10)
df4.rename(columns={'Coef':'CoefTextPOS','Feature':'TextPOS'}, inplace=True)
df4['TextPOS']=df4.apply(lambda x: text_pos_inv_convert(x['TextPOS']), axis=1)
df4.reset_index(inplace=True)
df_kk_top_features = pd.concat([df1,df2,df3,df4],axis=1)
df_kk_top_features.drop('index', axis=1, inplace=True)
df_kk_top_features

,Author,CoefChar,Char,CoefWord,Word,CoefText,Text,CoefTextPOS,TextPOS
0,KimKardashian,-0.914739,url,-0.930364,url,-0.930364,url,-1.297214,NNP-NNP-NNP
1,KimKardashian,-0.946238,re,-1.029619,the,-0.930364,i,-1.823307,IN-NNP-NNP
2,KimKardashian,-0.962367,he,-1.083203,ref,-1.029619,the,-2.085670,NN-NN-NNP
3,KimKardashian,-0.978761,ur,-1.443818,to,-1.083203,ref,-2.110988,NN-NNP-NNP
4,KimKardashian,-1.003867,ing,-1.511871,my,-1.443818,to,-2.110988,NN-IN-NNP
5,KimKardashian,-1.047163,to,-1.842725,in,-1.511871,my,-2.136963,NNP-NN-NN
6,KimKardashian,-1.074071,ng,-1.862527,so,-1.804258,a,-2.191030,IN-DT-NN
7,KimKardashian,-1.083203,ref,-1.924402,this,-1.823307,s,-2.219201,NNP-NNP-NN
8,KimKardashian,-1.111112,ef,-1.924402,is,-1.842725,in,-2.248189,NN-NN-NN
9,KimKardashian,-1.355710,is,-1.945909,for,-1.862527,so,-2.248189,DT-NN-IN


In [62]:
author_predicted=pd.DataFrame(author_predicted,columns=['predicted'])
df_wrong_result = pd.concat([twt_test.reset_index(),author_test.reset_index(),author_predicted], axis=1)
df_wrong_result.drop('index', axis=1, inplace=True)
df_wrong_result.drop('text_pos', axis=1, inplace=True)
df_wrong_result=df_wrong_result[df_wrong_result['author']<>df_wrong_result['predicted']]
df_wrong_result.head(10)

,text,author,predicted
0,New York State of mind URL,KimKardashian,HillaryClinton
17,A wise woman once told me this...her name migh...,KimKardashian,HillaryClinton
124,I’m #BreakingUpWith my trainer because I only ...,KimKardashian,HillaryClinton
134,Every time u tweet #BeatCancer EBay & Miller/C...,KimKardashian,HillaryClinton
146,RT REF Live without pretending. Love without d...,KimKardashian,HillaryClinton
150,"Thank you, Texas! #SuperTuesdaypic.twitter.com...",HillaryClinton,KimKardashian
180,A very Happy Birthday to my very wonderful hus...,HillaryClinton,KimKardashian
185,So pleased & proud to see Mayor REF sworn in t...,HillaryClinton,KimKardashian
194,I don't remember posting these Jerusalem pix.T...,KimKardashian,HillaryClinton
207,"""You can go vote early right now. Early voting...",HillaryClinton,KimKardashian
